In [248]:
import numpy as np
import numpy
from nbodykit.lab import BigFileMesh, BigFileCatalog
from pmesh.pm import ParticleMesh
import os, sys
import matplotlib.pyplot as plt
%matplotlib inline
%load_ext autoreload
%autoreload 2

from nbodykit.cosmology import Cosmology, EHPower, Planck15

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [249]:
import tensorflow as tf
# tf.enable_eager_execution()

In [250]:
sys.path.append('../code/utils/')
import tools
from background import *
from tfpm import *
from tfpmtools import *
from fpmfuncs import *
from fpm import *



In [251]:
state = fastpm()

/global/homes/c/chmodi/.conda/envs/fastpm/lib/python3.6/site-packages/nbodykit/cosmology/power/linear.py:228: FutureWarning: NoWiggleEHPower is deprecated. Use LinearPower with transfer set to 'EisensteinHu'
  warnings.warn("NoWiggleEHPower is deprecated. Use LinearPower with transfer set to 'EisensteinHu'", FutureWarning)


Step F 0.1000 - (0.1000) -> 0.1000 S 0.1000 P 0.1000 F 0.1000
Step K 0.1000 - (0.1000) -> 0.1803 S 0.1000 P 0.1803 F 0.1000
Step D 0.1000 - (0.1803) -> 0.3250 S 0.3250 P 0.1803 F 0.1000
Step F 0.1000 - (0.3250) -> 0.3250 S 0.3250 P 0.1803 F 0.3250
Step K 0.1803 - (0.3250) -> 0.3250 S 0.3250 P 0.3250 F 0.3250
Step K 0.3250 - (0.3250) -> 0.4228 S 0.3250 P 0.4228 F 0.3250
Step D 0.3250 - (0.4228) -> 0.5500 S 0.5500 P 0.4228 F 0.3250
Step F 0.3250 - (0.5500) -> 0.5500 S 0.5500 P 0.4228 F 0.5500
Step K 0.4228 - (0.5500) -> 0.5500 S 0.5500 P 0.5500 F 0.5500
Step K 0.5500 - (0.5500) -> 0.6529 S 0.5500 P 0.6529 F 0.5500
Step D 0.5500 - (0.6529) -> 0.7750 S 0.7750 P 0.6529 F 0.5500
Step F 0.5500 - (0.7750) -> 0.7750 S 0.7750 P 0.6529 F 0.7750
Step K 0.6529 - (0.7750) -> 0.7750 S 0.7750 P 0.7750 F 0.7750
Step K 0.7750 - (0.7750) -> 0.8803 S 0.7750 P 0.8803 F 0.7750
Step D 0.7750 - (0.8803) -> 1.0000 S 1.0000 P 0.8803 F 0.7750
Step F 0.7750 - (1.0000) -> 1.0000 S 1.0000 P 0.8803 F 1.0000
Step K 0

# Check lptz0

In [252]:
bs = 100
nc = 32
pm = ParticleMesh(BoxSize=bs, Nmesh = [nc, nc, nc], dtype='f4')
grid = pm.generate_uniform_particle_grid(shift=0).astype(np.float32)
kvec = tools.fftk((nc, nc, nc), bs, dtype=np.float32)
gdict = {'bs':bs, 'nc':nc, 'kvec':kvec, 'ndim':3}


whitec = pm.generate_whitenoise(100, mode='complex', unitary=False)
lineark = whitec.apply(lambda k, v:Planck15.get_pklin(sum(ki ** 2 for ki in k)**0.5, 0) ** 0.5 * v / v.BoxSize.prod() ** 0.5)
linear = lineark.c2r()
tlinear = tf.constant(linear[...])
tlineark = tf.multiply(tf.spectral.rfft3d(tlinear), 1/nc**3)



/global/homes/c/chmodi/.conda/envs/fastpm/lib/python3.6/site-packages/ipykernel/__main__.py:9: DeprecationWarning: mode argument is deprecated, use type


In [253]:
lptdisp = lptz0(lineark, grid, order=2)
lptpos = grid + lptdisp
lptmesh =  pm.paint(lptpos)


In [ ]:
# #
# tlinear = tf.constant(linear[...])
# tlineark = tf.multiply(tf.spectral.rfft3d(tlinear), 1/nc**3)
tlptmesh = tf.zeros_like(tlinear)
wt = tf.ones(nc**3)
# wt = np.ones(nc**3, dtype=np.float32)
tlptdisp = tflptz0(tlineark, grid, gdict, order=2)
tlptpos = tf.add(grid, tlptdisp)
tlptmesh = cic_paint(tlptmesh, tf.multiply(tlptpos, nc/bs), weight=wt)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    tflptpos, tflptmesh = sess.run([tlptpos, tlptmesh])

In [ ]:
tflptpos-lptpos

array([[ 2.3841858e-06, -1.1920929e-07, -1.1920929e-06],
       [ 2.8610229e-06,  0.0000000e+00, -1.9073486e-06],
       [ 9.5367432e-07,  2.3841858e-06, -9.5367432e-07],
       ...,
       [ 0.0000000e+00,  0.0000000e+00,  7.6293945e-06],
       [ 0.0000000e+00,  0.0000000e+00,  7.6293945e-06],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00]], dtype=float32)

In [ ]:
fig, axar = plt.subplots(2, 2, figsize = (11, 11))
ax = axar[0]
im = ax[0].imshow(linear[...].sum(axis=0))
plt.colorbar(im, ax=ax[0])
ax[0].set_title('Linear')
im = ax[1].imshow((lptmesh-tflptmesh)[...].sum(axis=0))
plt.colorbar(im, ax=ax[1])
ax[1].set_title('Difference 2LPT')
ax = axar[1]
im = ax[0].imshow(tflptmesh[...].sum(axis=0))
plt.colorbar(im, ax=ax[0])
ax[0].set_title('TF 2LPT')
im = ax[1].imshow(lptmesh[...].sum(axis=0))
plt.colorbar(im, ax=ax[1])
ax[1].set_title('Yu 2LPT')



Text(0.5,1,'Yu 2LPT')

# FastPM

### lpt IC

In [95]:
conf = Config()
solver = Solver(pm, conf['cosmology'])

In [181]:
a0, a1 = conf['stages'][2], conf['stages'][3]
ah = (a0*a1)**0.5

In [163]:
state = solver.lpt(lineark, grid, a0, order=2)

In [164]:
# #
tflptic = tflptinit(tlineark, grid, a0, gdict, conf, order=2)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    ttmp = sess.run(tflptic)

In [165]:
(ttmp[0])/ state.S

array([[0.9999995 , 0.99999946, 0.9999995 ],
       [0.9999999 , 0.9999996 , 0.99999976],
       [0.99999964, 0.99999917, 0.99999976],
       ...,
       [0.9999903 , 0.9999975 , 0.99999857],
       [0.99999684, 0.9999993 , 0.99999696],
       [0.9999961 , 0.99999946, 1.0000015 ]], dtype=float32)

In [166]:
(grid+ttmp[0])/ state.X

array([[0.9999995 , 0.99999946, 0.9999995 ],
       [0.9999999 , 0.9999996 , 0.99999994],
       [0.99999964, 0.99999917, 1.        ],
       ...,
       [1.        , 1.        , 1.0000001 ],
       [1.        , 1.0000001 , 1.        ],
       [1.        , 1.        , 1.        ]], dtype=float32)

In [167]:
(ttmp[1])/ state.V

array([[0.001     , 0.001     , 0.001     ],
       [0.001     , 0.001     , 0.001     ],
       [0.001     , 0.001     , 0.001     ],
       ...,
       [0.00099999, 0.001     , 0.001     ],
       [0.001     , 0.001     , 0.001     ],
       [0.001     , 0.001     , 0.001     ]], dtype=float32)

In [168]:
(ttmp[2])/ state.F

array([[0.9999999 , 0.9999997 , 0.99999946],
       [1.0000002 , 0.99999976, 0.9999998 ],
       [0.99999976, 0.9999993 , 0.9999999 ],
       ...,
       [0.9999872 , 0.99999726, 0.9999986 ],
       [0.9999969 , 0.99999964, 0.9999968 ],
       [0.99999577, 0.99999964, 1.0000014 ]], dtype=float32)

### Kick

In [240]:
# #
tflptic = tflptinit(tlineark, grid, a0, gdict, conf, order=2)
tfkicked = tfKick(tflptic, a0, ah, a1, gdict, conf)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    ttmp0, ttmp1 = sess.run([tfkicked, tflptic])

In [241]:
ttmp1[2]

array([[-0.37545836, -0.3179639 ,  0.8406729 ],
       [-0.35242867, -0.68979555,  1.5721769 ],
       [-0.03175962, -1.0161371 ,  2.0074666 ],
       ...,
       [-0.14827621,  0.29194495, -1.5478739 ],
       [-0.14681464,  0.06391829, -0.33988973],
       [-0.38491547, -0.16939926,  0.43170738]], dtype=float32)

In [198]:
ii = 0 
ttmp0[ii]/ttmp1[ii]

array([[ 0.881663  ,  1.0864391 ,  1.1980553 ],
       [ 0.91098607,  1.1097639 ,  1.1650522 ],
       [ 0.993148  ,  1.1298778 ,  1.1570319 ],
       ...,
       [ 0.31644115, -2.9079506 ,  1.1418982 ],
       [ 0.9257526 ,  0.9769839 ,  1.1101732 ],
       [ 0.23444332,  1.0380741 ,  1.2646092 ]], dtype=float32)

In [195]:
# tsave = ttmp0 + ttmp1[0]
# tsave

array([[ 1.0890334 , -1.5558597 ,  1.97977   ],
       [ 1.404178  , -2.715118  ,  4.3203897 ],
       [ 1.792136  , -3.441487  ,  5.758417  ],
       ...,
       [ 0.02672304,  0.08457387, -4.8493495 ],
       [ 0.7126563 , -1.0562764 , -1.3333689 ],
       [ 0.04589054, -1.7980746 ,  0.8032265 ]], dtype=float32)

In [199]:
tsave / ttmp0[0]


array([[1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       ...,
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.]], dtype=float32)

### Force

In [247]:
# #
tflptic = tflptinit(tlineark, grid, a0, gdict, conf, order=2)
tfforced = tfForce(tflptic, a0, ah, a1, gdict, conf)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    ttmp0, ttmp1 = sess.run([tfforced, tflptic])

Tensor("Assign_103:0", shape=(3, 32768, 3), dtype=float32_ref) Tensor("Mul_3081:0", shape=(32768, 3), dtype=float32)
Tensor("Assign_103:0", shape=(3, 32768, 3), dtype=float32_ref) Tensor("Add_3810:0", shape=(32768, 3), dtype=float32)
Tensor("Assign_103:0", shape=(3, 32768, 3), dtype=float32_ref) Tensor("zeros_95:0", shape=(32768, 3), dtype=float32)


InvalidArgumentError: Must have updates.shape = indices.shape + params.shape[1:] or updates.shape = [], got updates.shape [32768,3], indices.shape [], params.shape [32,32,32]
	 [[{{node ScatterAdd_14}} = ScatterAdd[T=DT_FLOAT, Tindices=DT_INT32, use_locking=false, _device="/job:localhost/replica:0/task:0/device:CPU:0"](Assign_103, ScatterAdd_14/indices, Add_3810)]]

Caused by op 'ScatterAdd_14', defined at:
  File "/global/homes/c/chmodi/.conda/envs/fastpm/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/global/homes/c/chmodi/.conda/envs/fastpm/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/global/homes/c/chmodi/.conda/envs/fastpm/lib/python3.6/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/global/homes/c/chmodi/.conda/envs/fastpm/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/global/homes

In [ ]:
ttmp0[2], ttmp1[2]